In [ ]:
# Load from parent directory if not installed
import importlib
import os

if not importlib.util.find_spec("sammo"):
    import sys

    sys.path.append("../../")
os.environ["CACHE_FILE"] = "cache/quickstart.tsv"

# 🚀 Quick Start

At the core of SAMMO are symbolic prompt programs. This tutorial will show you a few simple programs.

To run this example, you need credentials to an AzureOpenAI API compatible model. 

In [ ]:
# %load -r 3:25 _init.py
import pathlib
import sammo
from sammo.runners import AzureChat
from sammo.base import Template, EvaluationScore
from sammo.components import Output, GenerateText, ForEach, Union
from sammo.extractors import ExtractRegex
from sammo.data import DataTable
import json
import requests
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider  

if not 'AZURE_OPENAI_ENDPOINT' in os.environ:
    raise ValueError("Please set the environment variable 'AZURE_OPENAI_ENDPOINT'.")

if not 'AZURE_OPENAI_ENDPOINT_DEPLOYMENT' in os.environ:
    raise ValueError("Please set the environment variable 'AZURE_OPENAI_ENDPOINT_DEPLOYMENT'.")
     
# Initialize Azure OpenAI Service client with Entra ID authentication
# similar as https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity#chat-completions
token_provider = get_bearer_token_provider(  
    DefaultAzureCredential(),  
    "https://cognitiveservices.azure.com/.default"  
)  
  
api_config = {
    "azure_ad_token_provider": token_provider,
    "endpoint": os.environ["AZURE_OPENAI_ENDPOINT"],
    "api_version": "2024-05-01-preview",  # set to whatever version needed
    "deployment_id": os.environ["AZURE_OPENAI_ENDPOINT_DEPLOYMENT"],
}

_ = sammo.setup_logger("WARNING")  # we're only interested in warnings for now

runner = AzureChat(
    model_id=api_config["deployment_id"],
    api_config=api_config,
    cache=os.getenv("CACHE_FILE", "cache.tsv"),
    timeout=30,
)

Let's write our first symbolic prompt program (SPP)! How about a quick 'Hello World?'?

In [ ]:
spp_hello_world = Output(GenerateText("Hello World!"))
spp_hello_world.run(runner)

## Writing symbolic prompt programs
Okay, let's move on to a more interesting example. For a list of countries, we want the top reason to visit:

In [ ]:
COUNTRIES = ["Switzerland", "Morocco", "Tanzania", "Indonesia", "Peru"]

reason_to_visit = GenerateText(
    Template("What is the top reason to visit {{input}} in one sentence?")
)
Output(reason_to_visit).run(runner, COUNTRIES)[:2]

What happens under the hood is that SAMMO parallizes the execution across all inputs automatically! 

Let's add the best time to visit to it and combine both pieces of information.

In [ ]:
when_to_visit = GenerateText(
    Template(
        "Which season is the best time to visit {{input}}? Answer in one sentence."
    )
)
country_pages = Template(
    "# {{input}}\n{{reason}}\n\n## When to Visit\n{{when}}",
    reason=reason_to_visit,
    when=when_to_visit,
)
written_pages = Output(country_pages).run(runner, COUNTRIES)
written_pages[:2]

Nice! To see what operations SAMMO executed under the hood, we can use `plot_call_trace()` to show all intermediate calls. You can click on each node to get more info.

In [ ]:
written_pages.outputs[0].plot_call_trace()

## Recap
Let's talk about some of the key concepts from SAMMO we have used:

1. We constructed a **symbolic prompt program** — a dynamic prompt that is re-used for different inputs.
2. This SPP has a structure which was constructed by nesting **components** from SAMMO.
3. To get the **output**, we call `run()` on the Output component which returns a DataTable.
4. SAMMO **parallelized** execution for us on the input data — no extra work was needed! 